<a href="https://colab.research.google.com/github/derek-shing/LAParking/blob/master/LAParking.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [33]:
# Loading Parking space table

import requests
import json
import pandas as pd
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
endpoint = 'https://data.lacity.org/resource/s49e-q6j2.json'
data = requests.get(endpoint).json()

In [0]:
df = pd.DataFrame(data)

In [0]:
df_csv = pd.read_csv('/content/drive/My Drive/Parking_Meter_Inventory.csv')

In [35]:
df_csv.shape

(33989, 9)

In [0]:
occupid_api = 'https://data.lacity.org/resource/e7h6-4a3e.json'
occupid_data = requests.get(occupid_api).json()

In [0]:
df_occ = pd.DataFrame(occupid_data)

In [0]:
df_csv = df_csv.rename(columns={'SpaceID':'spaceid'})

In [46]:
df_csv.head()

,spaceid,BlockFace,MeterType,RateType,RateRange,MeteredTimeLimit,ParkingPolicy,StreetCleaning,LatLng
0,WU150,501 UNION DR,Single-Space,FLAT,$1.00,2HR,8A-6P Mon-Sat,12P-3P Mon,"(34.057219, -118.268751)"
1,MV161,12200 VENICE BL,Single-Space,FLAT,$1.00,2HR,8A-8P Mon-Sat,NaN,"(34.0043, -118.432007)"
2,ER138,2000 COLORADO BL,Single-Space,FLAT,$1.00,2HR,8A-8P Mon-Sat,NaN,"(34.139067, -118.210994)"
3,BF66,7901 BEVERLY BL,Single-Space,FLAT,$1.00,2HR,"8A-8P Mon-Sat, 11A-8P Sun",NaN,"(34.076203, -118.362266)"
4,CB121,1301 S HOPE ST,Single-Space,TOD,$1 - $2,4HR,8A-8P Mon-Sat,NaN,"(34.038863, -118.26618)"


In [41]:
df_occ.head()

,eventtime,occupancystate,spaceid
0,2019-09-01T02:38:09.000,OCCUPIED,CT555
1,2019-09-01T02:38:02.000,VACANT,LT310
2,2019-09-01T02:38:00.000,OCCUPIED,WV223
3,2019-09-01T02:37:56.000,VACANT,WV216
4,2019-09-01T02:37:56.000,OCCUPIED,LT62


In [0]:
final_df=pd.merge(df_occ, df_csv, on='spaceid',how='left')

In [0]:
df.head()

In [30]:
df.shape

(1000, 15)

In [85]:
final_df.shape

(1000, 11)

In [86]:
final_df.head()

,eventtime,occupancystate,spaceid,BlockFace,MeterType,RateType,RateRange,MeteredTimeLimit,ParkingPolicy,StreetCleaning,LatLng
0,2019-09-01T02:38:09.000,OCCUPIED,CT555,700 NEW HIGH ST,Single-Space,TOD,$0.5 - $1.5,4HR,8A-8P Mon-Sat,4A-630A Thu,"(34.060982, -118.238105)"
1,2019-09-01T02:38:02.000,VACANT,LT310,300 BOYD ST,Single-Space,TOD,$0.5 - $4,2HR,PKG 8A-6P Mon-Sat | NP 7P-7A Nightly,NaN,"(34.046815, -118.243275)"
2,2019-09-01T02:38:00.000,OCCUPIED,WV223,10900 WEYBURN AV,Single-Space,TOD,$1 - $2,2HR,"8A-8P Mon-Thu, 8A-12A Fri-Sat, 11A-8P Sun",NaN,"(34.062496, -118.445753)"
3,2019-09-01T02:37:56.000,VACANT,WV216,10900 WEYBURN AV,Single-Space,TOD,$1 - $2,2HR,"8A-8P Mon-Thu, 8A-12A Fri-Sat, 11A-8P Sun",NaN,"(34.062507, -118.446446)"
4,2019-09-01T02:37:56.000,OCCUPIED,LT62,301 E 2ND ST,Single-Space,TOD,$0.5 - $4,2HR,8A-8P Mon-Sat,NaN,"(34.048229, -118.239982)"


In [0]:
occupied_filter= final_df['occupancystate']=='OCCUPIED'
vacant_filter= final_df['occupancystate']=='VACANT'

In [55]:
final_df[occupied_filter].shape

(506, 11)

In [58]:
final_df[vacant_filter].head()

,eventtime,occupancystate,spaceid,BlockFace,MeterType,RateType,RateRange,MeteredTimeLimit,ParkingPolicy,StreetCleaning,LatLng
1,2019-09-01T02:38:02.000,VACANT,LT310,300 BOYD ST,Single-Space,TOD,$0.5 - $4,2HR,PKG 8A-6P Mon-Sat | NP 7P-7A Nightly,NaN,"(34.046815, -118.243275)"
3,2019-09-01T02:37:56.000,VACANT,WV216,10900 WEYBURN AV,Single-Space,TOD,$1 - $2,2HR,"8A-8P Mon-Thu, 8A-12A Fri-Sat, 11A-8P Sun",NaN,"(34.062507, -118.446446)"
7,2019-09-01T02:37:49.000,VACANT,CB3930,400 S WALL ST,Single-Space,TOD,$0.5 - $1.5,4HR,PKG 8A-6P Mon-Sat | NP 7P-7A Nightly,NaN,"(34.045013, -118.245596)"
10,2019-09-01T02:37:43.000,VACANT,CB3235,400 W. 8TH ST,Single-Space,FLAT,$4.00,4HR,8A-8P Mon-Sat,NaN,"(34.044706, -118.25629)"
13,2019-09-01T02:37:40.000,VACANT,LT60A,301 E 2ND ST,Single-Space,TOD,$0.5 - $4,2HR,8A-8P Mon-Sat,NaN,"(34.048154, -118.239868)"


In [0]:
import folium
from folium import plugins


In [13]:
df['latlng'][0]

{'latitude': '34.057219', 'longitude': '-118.268751'}

In [0]:
x = df['latlng'][0]['latitude']
y = df['latlng'][0]['longitude']

In [0]:
x = float(x)
y = float(y)

In [0]:
final_df = final_df.dropna(axis=0, subset=['LatLng'])

In [0]:
from ast import literal_eval as make_tuple

In [76]:
make_tuple(final_df[occupied_filter]['LatLng'][0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  """Entry point for launching an IPython kernel.


(34.060982, -118.238105)

In [0]:
final_df['LatLng'] = final_df['LatLng'].apply(make_tuple)

In [91]:
final_df['LatLng'][0][1]

-118.238105

In [0]:
parking_space_list=[]
for point in final_df[occupied_filter]['LatLng']:
  point=(,float(point['longitude']))
  parking_space_list.append(point)

In [25]:
len(parking_space_list)

1000

In [0]:
# Create a map using the Map() function and the coordinates for Boulder, CO
m = folium.Map(location=[x, y])

# Display the map
m

In [0]:
for point in final_df['LatLng'][occupied_filter]:
    folium.Marker(point).add_to(m)

In [96]:
m